# Reinforcement Learning, an introduction, Sutton and Barto

My open questions:
* find example of MDP where optimal eps-greedy policy has value faunction such that the grredy policy with respect to those values is not optimal (in the general sense).


## Chapter 2. Multi-arm Bandits

You are faced repeatedly with a choice among $k$ different options or actions. How do you choose to maximize your return over long term?

Notation:
- $A_t$ - action at time t
- $R_t$ - reward after taking action $A_t$
- $q_*(a)=E[R_t|A_t=a]$ - value of action $a$
- $Q_t(a)$ - estimate of $q_*(a)$ at time t

**Action-value methods**
Natural way to estimate $Q(a)$ would be to compute **sample average**:
$$Q_t(a) = \frac{\text{sum of rewards when a taken prior to t}}{\text{number of times a taken prior to t}}$$

and then choose the arm with the biggest estimate (**greedy action**). 

Incremental implementation:

$$Q_{n+1} = Q_n + \frac{1}{n}[R_n-Q_n]$$

For non-stationary problems better might be (**recency**) **weighted average**:
$$Q_{n+1} = Q_n + \alpha[R_n-Q_n]$$

The fundamental problem of multiarmed bandits is the **exploration vs exploitation ratio**, which is a way of choosing if we explore or exploit in a given step. Some approaches:

- $\epsilon$-greedy - choose random arm $\epsilon$ times, and greedy one $1-\epsilon$ times,
- optimistic initial vales - you start with some big values for each arm and then choose greedily and update them as you go (this way each arm will be choosen lots of times),
- Upper Confidence Bound, you choose according to arm value:
$$A_t=\arg\max_a[Q_t(a)+c\sqrt{\frac{\log{t}}{N_t(a)}}]$$
where $c$ is a constant that controls degree of exploration.
- gradient bandit algorithms - here we estimate a **policy** directly, which is a probability distribution on arms:
$$\pi_t(a) = P(A_t=a) = \frac{e^{H_t(a)}}{\sum_{b=1}^{k}e^{H_t(b)}}$$
where $H_t(a)$ are computed according to an update rule which is after selecting arm $A_t$ does:
$$H_{t+1}(A_t)=H_t(A_t)+\alpha(R_t-\overline{R_t})(1-\pi_t(A_t))$$ and
$$H_{t+1}(a)=H_t(a)-\alpha(R_t-\overline{R_t})\pi_t(a)$$ for $a\neq A_t$. It can be shown that the rule above can be derived from:
$$H_{t+1}(a)=H_t(a)+\alpha \frac{\partial E[R_t]}{\partial H_t(a)}$$
The substraction of the term $\overline{R_t}$ makes the variance lower (but could be skipped).



## Chapter 3. Finite Markov Decision Processes

This chapter basically introduces notation and sets up the problem for future chapters.

At each time step $t=1,2,3,...$ the agent receives some representation of the environment's **state**, $S_t\in\mathcal{S}$, where $\mathcal{S}$ is a set of all possible states. Then, the agent chooses some **action** $A_t\in\mathcal{A}(S_t)$, where $\mathcal{A}(S_t)$ is a set of actions available in the state $S_t$. One time-step later agent receives a numerical **reward** (as a part of consequence), $R_{t+1} \in \mathcal{R} \subset\mathbb{R}$ and finds itself in new state $S_{t+1}$.

At each step, agent implements a mapping from states to distributions over actions which is called a **policy** and is denoted $\pi_t$, where $\pi_t(a|s)$ is the probability that $A_t=a$ if $S_t=s$.

The **return** is a sum of rewards over time:
$$G_t=\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}$$
where $\gamma\in[0,1]$ and is usually equal to $1$ for **episodic tasks** (such ones that the probability that they will end is $1$ and $R_t$ will be always $0$ after some time $T$) and less then $1$ for **continuing tasks**.

The environment has **the Markov property** when it's response at time $t+1$ depends only on it's state and action taken at time $t$:
$$\mathbb{P}(S_{t+1},R_{t+1}|S_0,A_0,R_1,...,S_{t-1},A_{t-1},R_t,S_t,A_t) = \mathbb{P}(S_{t+1},R_{t+1}|S_t,A_t)$$

A reinforcement learning task that satisfies the Markov property is called a **Markov decision process** (MDP). If the state and action spaces are finite then it's called a **finite** MDP.

Some more notation:

$$p(s',r|s,a)=\mathbb{P}(S_{t+1}=s',R_{t+1}=r|S_t=s,A_t=a)$$
$$r(s,a) = \mathbb{E}[R_{t+1}|S_t=s,A_t=a] = \sum_{r\in\mathcal{R}} r\sum_{s'\in\mathcal{S}} p(s',r|s,a)$$

**Value function** for policy $\pi$:
$$v_\pi(s) = \mathbb{E}_\pi[G_t|S_t=s]=\mathbb{E}_\pi[\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}|S_t=s]$$
**Action-value function** for policy $\pi$:
$$q_\pi(s,a) = \mathbb{E}_\pi[G_t|S_t=s,A_t=a]=\mathbb{E}_\pi[\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}|S_t=s,A_t=a]$$

Bellman equation for value function:
$$v_\pi(s)=\sum_a\pi(a|s)\sum_{s',r}p(s',r|s,a)(r+\gamma v_\pi(s'))$$

TODO

## Chapter 4. Dynamic Programming

**Key idea of DP:** Use value functions to organize search for good policies.

**Key characteristics:**
* complete knowledge of environment - knowing the transition probabilities $p(s',r|s,a)$

Observations:
* **Iterative policy evaluation:** We can use Bellman equations to compute value function for given policy $\pi$: 
$$v_{k+1}(s)=\mathbb{E}_{\pi}[R_{t+1}+\gamma v_k(S_{t+1})| S_t=s] = \sum_{a}\pi(a|s)\sum_{s',r}p(s',r|s,a)[r+\gamma v_k(s')]$$
* once we have a value function we can construct new policy that is greedy with respect to the value function:
    * if resulting policy is the same as the initial one - great! We have found the optimal policy.
    * if it's different then lets find its value function (it's certainly not worse because of policy improvement theorem - see below).
* **Policy improvement theorem:**
$$\forall_s q_{\pi}(s, \pi'(s)) \geq v_{\pi}(s) \Rightarrow \forall_s v_{\pi'}(s)\geq v_{\pi}(s)$$
* **Policy iteration** is the procedure described above.
* It is often not necessary for the value function to converge for the greedy policy with respect to it to be better then the original one. Based on this observation is **value iteration**:
$$v_{k+1}(s)= \max_a\sum_{s',r}p(s',r|s,a)[r+\gamma v_k(s')]$$
After it converges, the policy that is greedy with respect to it will be optimal.
* it is not necessary to update values of all states in one sweep. In fact even if we update some states more often then the others, as long as all states get updates infinitely many times in the limit, the algorithm will converge. Such procedure is called **asynchronous dynamic programming**.
* **Generalized policy iteration** is an idea of letting policy evaluation (finding its value function) and policy improvement (finding better policy based on value function for current policy) interact.
* DP methods are polynomial in the number of states and actions.
* DP methods assume complete knowledge of the MDP.
* **In place updates** are ones in which only one set/table of values is used to do updates. This means that depending on the order some values will be updated using already new other values updated just before.
* **Bootstrapping** - an idea of updating estimates based on other estimates.


## Chapter 5. Monte Carlo Methods

Monte Carlo methods are based only on experience. They don't require a model and do not bootstrap. If you have a model then you can simulate experience. The basic idea is (for episodic tasks): simulate episodes and estimate state-action values as means of observed returns.

Observations:
* with no model of the environment, given a state, we don't know which actions will lead us to which further states, so what we estimate will be state-action values (not state values)! (Based on values we still wouldn't know which action to choose.)
* **soft policy** is a policy that selects all actions in all states with non-zero probability.
* **off-policy learning** is the kind of learning where policy used to generate episodes (**behaviour policy**) is defferent then the evaluated and improved one (**target policy**).
* when we do off-policy learning then we need to make adjustments to state-action values we are learning based on the fact that behaviour policy chooses action with different frequencies then target policy. Those adjusments are called **importance sampling**. There two basic (and other more complex ones) ways of doing it:

    path weights: $$\rho_t^T=\frac{\prod_{k=t}^{T-1}\pi(A_k|S_k)p(S_{k+1}|S_k, A_k)}{\prod_{k=t}^{T-1}\mu(A_k|S_k)p(S_{k+1}|S_k, A_k)}=\prod_{k=t}^{T-1}\frac{\pi(A_k|S_k)}{\mu(A_k|S_k)}$$

    **ordinary importance sampling**: $$V(s)=\frac{\sum_{t\in \mathcal{T}(s)}\rho_t^{T(t)}G_t}{|\mathcal{T}(s)|}$$ 
    
    **weighted importance sampling**: $$V(s)=\frac{\sum_{t\in \mathcal{T}(s)}\rho_t^{T(t)}G_t}{\sum_{t\in \mathcal{T}(s)}\rho_t^{T(t)}}$$
    
    where we keep numbering time across episodes (if first ends at time 100 then next begins at time 101), $\mathcal{T}(s)$ is set of all time steps in which state $s$ was visited and $T(t)$ is the number of final step in episode in which time $t$ occured.
* in the final algorithm we estimate state action values and based on algorithm from page 119 this is the estimate (with weighted importance sampling):
    $$Q(s,a)=\frac{\sum_{t\in \mathcal{T}(s)}\rho_{t+1}^{T(t+1)}G_t}{\sum_{t\in \mathcal{T}(s)}\rho_{t+1}^{T(t+1)}}$$
    
    I would compute it like this:
    $$Q(s,a)=\frac{\sum_{t\in \mathcal{T}(s)}R_t + \gamma V(S_{t+1})}{|\mathcal{T}(s)|}$$
    
    where $V(S_{t+1})$ is the weighted importance sampling version of value estimation given above. Hmm, **the difference is only in the way to compute expected rewards from taking action $a$ in state $s$**. In first case we have weighted average and in the other sample average.
    
* Ordinary importance sampling is unbiased but it has potentially unlimited (even infinite) varinace. Weighted importance sampling is asymptotically unbiased and has finite limited varinace (if we have bounded returns then the variance converges to zero). In practice the latter is preferred.
* In the perfect scenario we are able to do **exploring starts** which means we can start an episode in arbitrary state taking arbitrary action. Then for policy evaluation we compute sample returns from exploring starts. To find optimal policy we interleave policy evaluation and improvement (we can speed it up by doing policy improvements after each episode). This procedure is called **Monte Carlo ES**. **Caveat:** it has not been formally proven that this procedure converges to optimal policy.

Questions:
* the state-action values we are learning are non-stationary, because we do value and policy iteration. (It becomes stationary at some point) Then why not use weighted averages instead of sample averages (like in bandits for non-stationary problems). Hmm, if it becomes stationary then it doesn't matter in the limit.
* how to explore smartly? Using the info we have so far (kind of using UCB)? Maybe limiting amount of exploration moves within a path?
* maybe some way to do approx of exploring starts?
* how to improve on long paths? (For episodic tasks. Unless you are punished for long episodes.)